# BioMapAI Tutorial

This tutorial will guide you through:
1. **Environment Setup**  
2. **Data Preparation**  
3. **Model Initialization and Training**  
4. **Prediction and Evaluation**  
5. **Score Conversion and Weight Adjustment**  

By the end, you should be able to train your own model on custom data.


## 1. Environment Setup

1. **Download or Clone** the repository/project that contains:
   - **BioMapAI.py** (the script with model classes and methods)
   - **example_data/** (containing your `train_data.csv` and `test_data.csv`)
   - A script or notebook for your tutorial (this file)

2. **Install Required Libraries** (if you haven’t already):
```bash
pip install numpy pandas tensorflow
```

3. **Import modules** in Python:
```python
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import importlib.util
import os

# Clear any previous TensorFlow session
tf.keras.backend.clear_session()
```

4. **Function to import `BioMapAI.py`**:
```python
def import_module_with_full_path(file_path):
    base_filename = os.path.basename(file_path)
    module_name = os.path.splitext(base_filename)[0]
    module_spec = importlib.util.spec_from_file_location(module_name, file_path)
    imported_module = importlib.util.module_from_spec(module_spec)
    module_spec.loader.exec_module(imported_module)
    return imported_module
```


## 2. Data Preparation

Let’s assume you have:
- A **training dataset** named `train_data.csv`
- A **test dataset** named `test_data.csv`

Each row might look like:

| y0 (label) | Y1 | Y2 | ... | Y12 | X1 | X2 | ... | Xn |
|------------|----|----|-----|-----|----|----|-----|----|
| 0 or 1     | …  | …  | …   | …   | …  | …  | …   | …  |

Where:
- **y0** is your single-class label (e.g., 0 or 1)
- **Y** columns (Y1–Y12) are intermediate targets
- **X** columns (X1–Xn) are input features

### Load Your Data
```python
# 1) Import the BioMapAI module
BioMapAI = import_module_with_full_path("BioMapAI.py")

# 2) Load training data
data = pd.read_csv("example_data/train_data.csv", index_col=0)
y0 = data.iloc[:, 0]       # Single-class label
Y = data.iloc[:, 1:13]     # Intermediate target columns
X = data.iloc[:, 13:]      # Input features

# 3) Load test data
test_data = pd.read_csv("example_data/test_data.csv", index_col=0)
y0_test = test_data.iloc[:, 0]
Y_test = test_data.iloc[:, 1:13]
X_test = test_data.iloc[:, 13:]
```


## 3. Model Initialization and Training

### 3.1 **OmicScoreModel**
The **OmicScoreModel** is trained to predict the intermediate targets `Y`.

```python
model, model_history = BioMapAI.OmicScoreModel(
    epochs=500, 
    optimizer=tf.keras.optimizers.Adam(0.0005), 
    batch_size=64, 
    kernel_regularizer=tf.keras.regularizers.L2(0.008), 
    model_name="example"
).train(X, Y)
```
- **epochs**: Number of training iterations.
- **optimizer**: Choice of optimizer and learning rate.
- **batch_size**: Batch size for training.
- **kernel_regularizer**: Regularization to avoid overfitting.
- **model_name**: Useful if you want to save or track multiple models.

### 3.2 Monitoring the Training History
```python
print(model_history.history.keys())
# You can plot loss curves, accuracy, etc.
```


## 4. Prediction and Evaluation

### 4.1 Predict Intermediate Targets `Y`
```python
Y_pred = BioMapAI.OmicScoreModel().predict(model, X_test)
```
This outputs the intermediate predictions for each of the 12 `Y` columns.

### 4.2 Evaluate Intermediate Target Predictions
Compute the Mean Squared Error (MSE) on `Y_test`:
```python
mse = BioMapAI.OmicScoreModel().score(model, X_test, Y_test)
print("Intermediate Target MSE:", mse)
```


## 5. Score Conversion and Weight Adjustment

### 5.1 Build a Layer for Converting `Y` to `y0`
```python
Y_to_y_model = BioMapAI.ScoreLayer().build_model(Y, y0)
```
This creates a secondary model that maps the 12-dimensional `Y` to a single label or score.

### 5.2 Predict Final Labels `y0`
```python
y_pred = BioMapAI.ScoreYModel(model, Y_to_y_model).predict(X_test)
```
### 5.3 Evaluate Final Model
```python
loss, accuracy = BioMapAI.ScoreYModel(model, Y_to_y_model).evaluate(X_test, y0_test)
print("Final Model Loss:", loss)
print("Final Model Accuracy:", accuracy)
```

### 5.4 Adjusting Weights (Optional)
If you want to fine-tune how the intermediate predictions map to the final label:
```python
Y_to_y_model_adjust = BioMapAI.WeightsAdjust(model, X, Y, Y_to_y_model).adjust_score_weight()
```
Then re-predict and re-evaluate:
```python
y_pred = BioMapAI.ScoreYModel(model, Y_to_y_model).predict(X_test)
loss, accuracy = BioMapAI.ScoreYModel(model, Y_to_y_model).evaluate(X_test, y0_test)
print("Adjusted Final Model Loss:", loss)
print("Adjusted Final Model Accuracy:", accuracy)
```


## Putting It All Together

Below is a compact version of the entire flow. You can copy this code into its own file (e.g., `train_and_eval.py`) or run it as a Jupyter cell.


In [1]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
import random
import tensorflow as tf
tf.keras.backend.clear_session()

import importlib.util
import os

def import_module_with_full_path(file_path):
    base_filename = os.path.basename(file_path)
    module_name = os.path.splitext(base_filename)[0]
    module_spec = importlib.util.spec_from_file_location(module_name, file_path)
    imported_module = importlib.util.module_from_spec(module_spec)
    module_spec.loader.exec_module(imported_module)
    return imported_module

# 1) Import BioMapAI module
BioMapAI = import_module_with_full_path("BioMapAI.py")

# 2) Load data
data = pd.read_csv("example_data/train_data.csv", index_col=0)
y0 = data.iloc[:, 0]
Y = data.iloc[:, 1:13]
X = data.iloc[:, 13:]

test_data = pd.read_csv("example_data/test_data.csv", index_col=0)
y0_test = test_data.iloc[:, 0]
Y_test = test_data.iloc[:, 1:13]
X_test = test_data.iloc[:, 13:]

# 3) Train OmicScoreModel
model, model_history = BioMapAI.OmicScoreModel(
    epochs=500,
    optimizer=tf.keras.optimizers.Adam(0.0005),
    batch_size=64,
    kernel_regularizer=tf.keras.regularizers.L2(0.008),
    model_name="example"
).train(X, Y)

# 4) Predict and evaluate intermediate targets
Y_pred = BioMapAI.OmicScoreModel().predict(model, X_test)
mse = BioMapAI.OmicScoreModel().score(model, X_test, Y_test)
print("Intermediate Target MSE:", mse)

# 5) Build layer to convert Y --> y0
Y_to_y_model = BioMapAI.ScoreLayer().build_model(Y, y0)

# 6) Predict final output
y_pred = BioMapAI.ScoreYModel(model, Y_to_y_model).predict(X_test)

# 7) Evaluate final output
loss, accuracy = BioMapAI.ScoreYModel(model, Y_to_y_model).evaluate(X_test, y0_test)
print("Initial Final Model Loss:", loss)
print("Initial Final Model Accuracy:", accuracy)

# 8) (Optional) Adjust weights
Y_to_y_model_adjust = BioMapAI.WeightsAdjust(model, X, Y, Y_to_y_model).adjust_score_weight()

# 9) Predict & Evaluate again
y_pred = BioMapAI.ScoreYModel(model, Y_to_y_model).predict(X_test)
loss, accuracy = BioMapAI.ScoreYModel(model, Y_to_y_model).evaluate(X_test, y0_test)
print("Adjusted Final Model Loss:", loss)
print("Adjusted Final Model Accuracy:", accuracy)

2025-01-14 16:18:12.198394: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-14 16:18:13.159391: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/500
3/3 [==============================] - 5s 11ms/step - loss: 8.4408 - out0_loss: 0.2485 - out1_loss: 2.1312 - out2_loss: 1.3975 - out3_loss: 0.2631 - out4_loss: 0.1193 - out5_loss: 0.4596 - out6_loss: 0.1095 - out7_loss: 0.2369 - out8_loss: 0.3161 - out9_loss: 0.1983 - out10_loss: 0.4352 - out11_loss: 0.3197 - out0_mse: 0.6275 - out1_accuracy: 0.2057 - out2_accuracy: 0.4171 - out3_mse: 0.5669 - out4_mse: 0.2447 - out5_mse: 1.1690 - out6_mse: 0.2210 - out7_mse: 0.5565 - out8_mse: 0.8460 - out9_mse: 0.4066 - out10_mse: 1.1801 - out11_mse: 0.9427
Epoch 2/500
3/3 [==============================] - 0s 9ms/step - loss: 8.2360 - out0_loss: 0.2207 - out1_loss: 1.9974 - out2_loss: 1.5036 - out3_loss: 0.2788 - out4_loss: 0.1095 - out5_loss: 0.4855 - out6_loss: 0.0890 - out7_loss: 0.1775 - out8_loss: 0.3087 - out9_loss: 0.2135 - out10_loss: 0.3702 - out11_loss: 0.2837 - out0_mse: 0.5251 - out1_accuracy: 0.2686 - out2_accuracy: 0.3143 - out3_mse: 0.6294 - out4_mse: 0.2385 - out5_mse: 1.

3/3 [==============================] - 0s 8ms/step - loss: 7.3867 - out0_loss: 0.1652 - out1_loss: 1.9046 - out2_loss: 1.3402 - out3_loss: 0.2784 - out4_loss: 0.0817 - out5_loss: 0.3334 - out6_loss: 0.0739 - out7_loss: 0.1810 - out8_loss: 0.1985 - out9_loss: 0.1936 - out10_loss: 0.2916 - out11_loss: 0.2390 - out0_mse: 0.3507 - out1_accuracy: 0.3257 - out2_accuracy: 0.4571 - out3_mse: 0.5947 - out4_mse: 0.1658 - out5_mse: 0.9380 - out6_mse: 0.1479 - out7_mse: 0.3772 - out8_mse: 0.4727 - out9_mse: 0.4050 - out10_mse: 0.7191 - out11_mse: 0.5581
Epoch 16/500
3/3 [==============================] - 0s 8ms/step - loss: 7.2876 - out0_loss: 0.1965 - out1_loss: 1.7966 - out2_loss: 1.3537 - out3_loss: 0.2197 - out4_loss: 0.0881 - out5_loss: 0.3516 - out6_loss: 0.1030 - out7_loss: 0.1548 - out8_loss: 0.1937 - out9_loss: 0.2012 - out10_loss: 0.2709 - out11_loss: 0.2592 - out0_mse: 0.4496 - out1_accuracy: 0.3086 - out2_accuracy: 0.4400 - out3_mse: 0.4479 - out4_mse: 0.1767 - out5_mse: 1.0052 - out6_

3/3 [==============================] - 0s 8ms/step - loss: 6.5154 - out0_loss: 0.1321 - out1_loss: 1.5413 - out2_loss: 1.2674 - out3_loss: 0.2401 - out4_loss: 0.0653 - out5_loss: 0.3341 - out6_loss: 0.0798 - out7_loss: 0.1477 - out8_loss: 0.1512 - out9_loss: 0.1943 - out10_loss: 0.1895 - out11_loss: 0.1579 - out0_mse: 0.2753 - out1_accuracy: 0.3543 - out2_accuracy: 0.4800 - out3_mse: 0.5005 - out4_mse: 0.1307 - out5_mse: 0.8885 - out6_mse: 0.1597 - out7_mse: 0.2970 - out8_mse: 0.3309 - out9_mse: 0.3968 - out10_mse: 0.4198 - out11_mse: 0.3480
Epoch 30/500
3/3 [==============================] - 0s 9ms/step - loss: 6.5907 - out0_loss: 0.1506 - out1_loss: 1.6539 - out2_loss: 1.2638 - out3_loss: 0.2556 - out4_loss: 0.0726 - out5_loss: 0.2524 - out6_loss: 0.0855 - out7_loss: 0.1655 - out8_loss: 0.1509 - out9_loss: 0.1806 - out10_loss: 0.1987 - out11_loss: 0.1521 - out0_mse: 0.3124 - out1_accuracy: 0.3429 - out2_accuracy: 0.4343 - out3_mse: 0.5310 - out4_mse: 0.1462 - out5_mse: 0.5778 - out6_

3/3 [==============================] - 0s 7ms/step - loss: 6.0419 - out0_loss: 0.1217 - out1_loss: 1.5270 - out2_loss: 1.2003 - out3_loss: 0.2060 - out4_loss: 0.0642 - out5_loss: 0.2012 - out6_loss: 0.0884 - out7_loss: 0.1429 - out8_loss: 0.1192 - out9_loss: 0.1902 - out10_loss: 0.1613 - out11_loss: 0.0882 - out0_mse: 0.2487 - out1_accuracy: 0.4171 - out2_accuracy: 0.5200 - out3_mse: 0.4192 - out4_mse: 0.1287 - out5_mse: 0.4474 - out6_mse: 0.1782 - out7_mse: 0.2866 - out8_mse: 0.2572 - out9_mse: 0.3827 - out10_mse: 0.3420 - out11_mse: 0.1816
Epoch 44/500
3/3 [==============================] - 0s 7ms/step - loss: 6.0625 - out0_loss: 0.1238 - out1_loss: 1.5317 - out2_loss: 1.1726 - out3_loss: 0.1830 - out4_loss: 0.0570 - out5_loss: 0.2113 - out6_loss: 0.0762 - out7_loss: 0.1714 - out8_loss: 0.1623 - out9_loss: 0.1759 - out10_loss: 0.1396 - out11_loss: 0.1322 - out0_mse: 0.2517 - out1_accuracy: 0.3771 - out2_accuracy: 0.5257 - out3_mse: 0.3673 - out4_mse: 0.1142 - out5_mse: 0.4692 - out6_

...

3/3 [==============================] - 0s 7ms/step - loss: 1.6327 - out0_loss: 0.0191 - out1_loss: 0.2797 - out2_loss: 0.5770 - out3_loss: 0.0287 - out4_loss: 0.0130 - out5_loss: 0.0257 - out6_loss: 0.0177 - out7_loss: 0.0129 - out8_loss: 0.0124 - out9_loss: 0.0105 - out10_loss: 0.0247 - out11_loss: 0.0172 - out0_mse: 0.0382 - out1_accuracy: 0.9029 - out2_accuracy: 0.7714 - out3_mse: 0.0573 - out4_mse: 0.0260 - out5_mse: 0.0513 - out6_mse: 0.0355 - out7_mse: 0.0258 - out8_mse: 0.0249 - out9_mse: 0.0211 - out10_mse: 0.0495 - out11_mse: 0.0344
Epoch 478/500
3/3 [==============================] - 0s 7ms/step - loss: 1.6122 - out0_loss: 0.0171 - out1_loss: 0.3168 - out2_loss: 0.5311 - out3_loss: 0.0308 - out4_loss: 0.0117 - out5_loss: 0.0202 - out6_loss: 0.0175 - out7_loss: 0.0121 - out8_loss: 0.0131 - out9_loss: 0.0089 - out10_loss: 0.0238 - out11_loss: 0.0164 - out0_mse: 0.0341 - out1_accuracy: 0.8914 - out2_accuracy: 0.8000 - out3_mse: 0.0616 - out4_mse: 0.0235 - out5_mse: 0.0405 - out6

3/3 [==============================] - 0s 7ms/step - loss: 1.6127 - out0_loss: 0.0188 - out1_loss: 0.3565 - out2_loss: 0.5052 - out3_loss: 0.0277 - out4_loss: 0.0122 - out5_loss: 0.0257 - out6_loss: 0.0167 - out7_loss: 0.0126 - out8_loss: 0.0147 - out9_loss: 0.0100 - out10_loss: 0.0225 - out11_loss: 0.0162 - out0_mse: 0.0375 - out1_accuracy: 0.8971 - out2_accuracy: 0.7714 - out3_mse: 0.0555 - out4_mse: 0.0244 - out5_mse: 0.0514 - out6_mse: 0.0334 - out7_mse: 0.0252 - out8_mse: 0.0294 - out9_mse: 0.0200 - out10_mse: 0.0450 - out11_mse: 0.0323
Epoch 492/500
3/3 [==============================] - 0s 7ms/step - loss: 1.5612 - out0_loss: 0.0175 - out1_loss: 0.3019 - out2_loss: 0.5335 - out3_loss: 0.0227 - out4_loss: 0.0114 - out5_loss: 0.0198 - out6_loss: 0.0163 - out7_loss: 0.0112 - out8_loss: 0.0129 - out9_loss: 0.0094 - out10_loss: 0.0174 - out11_loss: 0.0146 - out0_mse: 0.0350 - out1_accuracy: 0.9029 - out2_accuracy: 0.7657 - out3_mse: 0.0455 - out4_mse: 0.0227 - out5_mse: 0.0397 - out6

6/6 [==============================] - 0s 1ms/step - loss: 0.3420 - accuracy: 0.9429
Epoch 24/200
6/6 [==============================] - 0s 1ms/step - loss: 0.3294 - accuracy: 0.9486
Epoch 25/200
6/6 [==============================] - 0s 1ms/step - loss: 0.3178 - accuracy: 0.9486
Epoch 26/200
6/6 [==============================] - 0s 1ms/step - loss: 0.3070 - accuracy: 0.9486
Epoch 27/200
6/6 [==============================] - 0s 1ms/step - loss: 0.2963 - accuracy: 0.9486
Epoch 28/200
6/6 [==============================] - 0s 1ms/step - loss: 0.2870 - accuracy: 0.9486
Epoch 29/200
6/6 [==============================] - 0s 1ms/step - loss: 0.2771 - accuracy: 0.9486
Epoch 30/200
6/6 [==============================] - 0s 1ms/step - loss: 0.2687 - accuracy: 0.9543
Epoch 31/200
6/6 [==============================] - 0s 1ms/step - loss: 0.2606 - accuracy: 0.9600
Epoch 32/200
6/6 [==============================] - 0s 1ms/step - loss: 0.2528 - accuracy: 0.9657
Epoch 33/200
6/6 [===============

6/6 [==============================] - 0s 1ms/step - loss: 0.0970 - accuracy: 0.9829
Epoch 107/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0967 - accuracy: 0.9829
Epoch 108/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0962 - accuracy: 0.9829
Epoch 109/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0958 - accuracy: 0.9829
Epoch 110/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0953 - accuracy: 0.9829
Epoch 111/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0950 - accuracy: 0.9829
Epoch 112/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0947 - accuracy: 0.9829
Epoch 113/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0942 - accuracy: 0.9829
Epoch 114/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0938 - accuracy: 0.9829
Epoch 115/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0935 - accuracy: 0.9829
Epoch 116/200
6/6 [=====

6/6 [==============================] - 0s 1ms/step - loss: 0.0802 - accuracy: 0.9829
Epoch 189/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0800 - accuracy: 0.9829
Epoch 190/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0799 - accuracy: 0.9829
Epoch 191/200
6/6 [==============================] - 0s 995us/step - loss: 0.0798 - accuracy: 0.9829
Epoch 192/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0797 - accuracy: 0.9829
Epoch 193/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0797 - accuracy: 0.9829
Epoch 194/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0795 - accuracy: 0.9829
Epoch 195/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0794 - accuracy: 0.9829
Epoch 196/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0794 - accuracy: 0.9829
Epoch 197/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0793 - accuracy: 0.9829
Epoch 198/200
6/6 [===

## Conclusion
You have now:
1. **Loaded and prepared** your data
2. **Trained** the OmicScoreModel on intermediate targets
3. **Transformed** those intermediate predictions to final labels
4. **Evaluated** and **(optionally) adjusted** model weights

Feel free to experiment with different architectures, hyperparameters (learning rate, batch size, etc.), or data splits. Modify **BioMapAI.py** to suit your specific needs.

**Happy modeling!**